### Tabla de informacion de juego 

Se cargan las librerias a usar 

In [20]:
#Carga de librerías
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

Carga de los datos

In [21]:
#CARGA DEL DATASET
data = pd.read_csv("game_info.csv", sep=',')

Primer vistaso a las columnas

In [22]:
pd.options.display.max_columns = 60
pd.options.display.max_columns

60

In [23]:
data.head(5)

,game_id,game_date,attendance,game_time
0,24600001,1946-11-01 00:00:00,NaN,NaN
1,24600003,1946-11-02 00:00:00,NaN,NaN
2,24600002,1946-11-02 00:00:00,NaN,NaN
3,24600004,1946-11-02 00:00:00,NaN,NaN
4,24600005,1946-11-02 00:00:00,NaN,NaN


Se comprueba que el dataset cargado tiene la cantidad de registros y columnas especificadas.

In [24]:
np.shape(data)

(58053, 4)

Se procedera a establecer un filtrado por fechas. Esto es para utilizar las temporadas siguientes (2020/2021) (2021/2022) (2022/2023) como fijan los parámetros del proyecto.

Dado que el final de la temporada anterior se retrasó debido la pandemia COVID-19, la NBA aprobó una fecha excepcional de inicio para el 22 de diciembre de 2020, cuando lo habitual era comenzar en el mes de octubre.

In [25]:
# Filtra los datos en fechas mayores a 2021-01-01.
data = data[data["game_date"]>'2020-12-31']
data = data[data["game_date"]<'2023-6-13']
np.shape(data)

(3527, 4)

In [26]:
data.tail(5)

,game_id,game_date,attendance,game_time
58048,42200403,2023-06-07 00:00:00,20019.0,2:24
58049,42200404,2023-06-09 00:00:00,20184.0,2:29
58050,42200405,2023-06-12 00:00:00,19537.0,2:22
58051,32200001,2023-02-19 00:00:00,17886.0,0:00
58052,32200001,2023-02-19 00:00:00,17886.0,0:00


### Se consultan valores nulos por primera vez.

In [27]:
# PORCENTAJE DE VALORES NULOS
nulos_total = data.isnull().sum()
porcentaje_nulos = (nulos_total / len(data)) * 100

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos_total,
    'Porcentaje Nulos': porcentaje_nulos
})

print(resumen_nulos)

            Valores Nulos  Porcentaje Nulos
game_id                 0          0.000000
game_date               0          0.000000
attendance            535         15.168699
game_time               0          0.000000


Observamos que tenemos un 15% de valores faltantes en la columna 'attendance'.

A continuación se fijara el parámetro por el cual se van a tratar los datos faltantes.

In [28]:
data['attendance'] = data['attendance'].fillna(data['attendance'].mean())

Se rellenaron los valores faltantes con la media de datos. 

Se consultan valores nulos nuevamente:

In [29]:
# PORCENTAJE DE VALORES NULOS
nulos_total = data.isnull().sum()
porcentaje_nulos = (nulos_total / len(data)) * 100

resumen_nulos = pd.DataFrame({
    'Valores Nulos': nulos_total,
    'Porcentaje Nulos': porcentaje_nulos
})

print(resumen_nulos)

            Valores Nulos  Porcentaje Nulos
game_id                 0               0.0
game_date               0               0.0
attendance              0               0.0
game_time               0               0.0


Controlamos que no existan más datos duplicados.

In [30]:
duplicados = data.duplicated(keep=False)

# Filtra las filas duplicadas
datos_duplicados = data[duplicados]
print(datos_duplicados)

        game_id            game_date  attendance game_time
58051  32200001  2023-02-19 00:00:00     17886.0      0:00
58052  32200001  2023-02-19 00:00:00     17886.0      0:00


Observamos que hay un valor duplicado, procedemos a eliminarlo mediante su indice.

In [31]:
data = data.drop(58052)

Ahora que no tenemos valores faltantes ni duplicados, procedemos a verificar el formato de las columnas.

In [32]:
print(data.dtypes)

game_id         int64
game_date      object
attendance    float64
game_time      object
dtype: object


Cambiamos el formato de la tabla de fechas a 'datetime' debido a que esta en 'object'.

In [33]:
data['game_date'] = pd.to_datetime(data['game_date'], errors='coerce')

In [34]:
data['game_time'] = pd.to_datetime(data['game_time'], format='%H:%M', errors='coerce').dt.time

Revisamos los cambios realizados:

In [35]:
print(data['game_date'].dtypes)

datetime64[ns]


In [36]:
data.head()

,game_id,game_date,attendance,game_time
54483,22000063,2020-12-31,15353.935829,02:27:00
54484,22000068,2020-12-31,15353.935829,02:07:00
54485,22000065,2020-12-31,15353.935829,02:24:00
54486,22000064,2020-12-31,15353.935829,02:02:00
54487,22000067,2020-12-31,15353.935829,02:05:00


In [37]:
np.shape(data)

(3526, 4)

### Ahora exportamos el dataset ya procesado.

In [38]:
data.to_csv('game_info_filtrado.csv', index=False)